## Stochastic Demand

### Main Questions
#### 1. How much inventory on hand (IOH) do I need so P[SO] ≤ some target service level?
#### 2. If I have a certain amount of inventory on hand, X, what is my P[SO]?
#### 3. Given a target service level or IOH, how many units do I expect to sell or be short?

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# pip install fitter
from fitter import Fitter, get_distributions

### Data Cleaning

In [ ]:
all_skus = pd.read_csv('online_retail2.csv')
all_skus = all_skus[all_skus.Quantity > 0]
all_skus = all_skus[all_skus.Price > 0]
all_skus.dropna(inplace= True)
all_skus.drop_duplicates(inplace=True)
# all_skus

### Select the sku and specify time units

In [ ]:
all_skus.StockCode.value_counts()

In [ ]:
sku = all_skus[all_skus.StockCode == '85123A']
sku = sku[['Quantity', 'InvoiceDate']].copy().reset_index(drop=True)
# sku

In [ ]:
sku.InvoiceDate = pd.to_datetime(sku.InvoiceDate)
# sku.InvoiceDate = sku.InvoiceDate.astype('datetime64')

In [ ]:
sku['Year'] = sku.InvoiceDate.dt.year.astype('str')
sku['Week'] = sku.InvoiceDate.dt.isocalendar().week.astype('str')
# sku

In [ ]:
sku.Week = [i if len(i)>1 else '0'+i for i in sku.Week]
sku['time_unit'] = sku.Year + '-' + sku.Week
sku.sample(10)

In [ ]:
grp = sku.groupby('time_unit').Quantity.sum().reset_index()
# grp

### Finding and eliminating outliers

In [ ]:
plt.figure(figsize=(12,6))
sns.boxplot(data=grp,x='Quantity');
# sns.boxenplot(data=grp,x='Quantity');

In [ ]:
grp = grp[grp.Quantity<2000].copy()
plt.figure(figsize=(12,6))
sns.boxplot(data=grp,x='Quantity');

### Find the best fitted distribution

In [ ]:
plt.figure(figsize=(10,5))
sns.histplot(data=grp, bins=50);

In [ ]:
demand_week = grp['Quantity'].values
# demand_week

In [ ]:
plt.figure(figsize=(12,6)) 
#Scipy
f = Fitter(data=demand_week)
f.fit()
f.summary()

In [ ]:
f.distributions

In [ ]:
f.get_best(method = 'sumsquare_error')

### Answering to the questions

In [ ]:
from scipy.stats import burr12
f.fitted_param['burr12']
burr12_dist = burr12(f.fitted_param['burr12'][0],f.fitted_param['burr12'][1],f.fitted_param['burr12'][2],f.fitted_param['burr12'][3])
burr12_dist

In [ ]:
mean_demand = burr12_dist.mean()
mean_demand

In [ ]:
std_demand = burr12_dist.std()
std_demand

#### 1- How much inventory on hand (IOH) do I need so P[SO] ≤ some target service level?


In [ ]:
CSL = 0.9
safety_stock = burr12_dist.ppf(q=CSL)
safety_stock

#### 2- If I have a certain amount of inventory on hand, X, what is my P[SO]?

In [ ]:
IOH = 1500
pr_short = 1 - burr12_dist.cdf(x=IOH)
pr_short

#### 3- Given a target service level or IOH, how many units do I expect to sell or be short?

In [ ]:
IOH = 1000
sold_less_than_IOH = burr12_dist.expect(lb=0,ub=IOH)

sold_more_than_IOH = (1 - burr12_dist.cdf(x=IOH)) * IOH

exp_sold = sold_less_than_IOH + sold_more_than_IOH
exp_sold

In [ ]:
IOH = 1000

def shortage(x):
    return x - IOH

short_less_than_IOH = burr12_dist.cdf(x=IOH) * 0

short_more_than_IOH = burr12_dist.expect(func=shortage,lb=IOH)

exp_short = short_less_than_IOH + short_more_than_IOH
exp_short

In [ ]:
exp_short + exp_sold

### Fit with most common distributions 

In [ ]:
plt.figure(figsize=(12,6))
f = Fitter(data=demand_week,
           distributions=['lognorm',
                          'norm',
                          'triang',
                          'gamma',
                          'uniform',
                          'expon',
                          'rayleigh']
           )
f.fit()
f.summary()

In [ ]:
f.distributions

In [ ]:
f.get_best(method = 'sumsquare_error')

### Answering to the questions

In [ ]:
from scipy.stats import rayleigh
f.fitted_param['rayleigh']
rayleigh_dist = rayleigh(f.fitted_param['rayleigh'][0],f.fitted_param['rayleigh'][1])
rayleigh_dist

In [ ]:
mean_demand = rayleigh_dist.mean()
mean_demand

In [ ]:
std_demand = rayleigh_dist.std()
std_demand

#### 1- How much inventory on hand (IOH) do I need so P[SO] ≤ some target service level?


In [ ]:
CSL = 0.9
safety_stock = rayleigh_dist.ppf(q=CSL)
safety_stock

#### 2- If I have a certain amount of inventory on hand, X, what is my P[SO]?

In [ ]:
IOH = 1500
pr_short = 1 - rayleigh_dist.cdf(x=IOH)
pr_short

#### 3- Given a target service level or IOH, how many units do I expect to sell or be short?

In [ ]:
IOH = 1000
sold_less_than_IOH = rayleigh_dist.expect(lb=0,ub=IOH)

sold_more_than_IOH = (1 - rayleigh_dist.cdf(x=IOH)) * IOH

exp_sold = sold_less_than_IOH + sold_more_than_IOH
exp_sold

In [ ]:
IOH = 1000

def shortage(x):
    return x - IOH

short_less_than_IOH = (rayleigh_dist.cdf(x=IOH)) * 0

short_more_than_IOH = rayleigh_dist.expect(func=shortage, lb=IOH)

exp_short = short_less_than_IOH + short_more_than_IOH
exp_short

In [ ]:
exp_short + exp_sold

### How to generate random numbers from the fitted distribution

In [ ]:
gen_demands = rayleigh_dist.rvs(size=100)
gen_demands = np.floor(gen_demands) + 1
gen_demands

In [ ]:
sns.histplot(x = gen_demands, bins=50, kde=True);